In [109]:
import pygrib
import pandas as pd
import numpy as np
import os
import tarfile
import logging
import sys
import requests
from datetime import datetime, timedelta

from pvlib import pvsystem, modelchain, location, irradiance
from pvlib.solarposition import get_solarposition
from pvlib import irradiance, solarposition

In [103]:
logging.basicConfig()
logger = logging.getLogger(__name__)
logger.setLevel(os.environ.get("LOG_LEVEL", logging.INFO))


class OpenDataAPI:
    def __init__(self, api_token: str):
        self.base_url = "https://api.dataplatform.knmi.nl/open-data/v1"
        self.headers = {"Authorization": api_token}

    def __get_data(self, url, params=None):
        return requests.get(url, headers=self.headers, params=params).json()

    def list_files(self, dataset_name: str, dataset_version: str, params: dict):
        return self.__get_data(
            f"{self.base_url}/datasets/{dataset_name}/versions/{dataset_version}/files",
            params=params,
        )

    def get_file_url(self, dataset_name: str, dataset_version: str, file_name: str):
        return self.__get_data(
            f"{self.base_url}/datasets/{dataset_name}/versions/{dataset_version}/files/{file_name}/url"
        )


def download_file_from_temporary_download_url(download_url, filename):
    try:
        with requests.get(download_url, stream=True) as r:
            r.raise_for_status()
            with open(filename, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
    except Exception:
        logger.exception("Unable to download file using download URL")
        sys.exit(1)

    logger.info(f"Successfully downloaded dataset file to {filename}")


def main():
    api_key = "eyJvcmciOiI1ZTU1NGUxOTI3NGE5NjAwMDEyYTNlYjEiLCJpZCI6ImE1OGI5NGZmMDY5NDRhZDNhZjFkMDBmNDBmNTQyNjBkIiwiaCI6Im11cm11cjEyOCJ9"
    dataset_name = "harmonie_arome_cy40_p1"
    dataset_version = "0.2"
    logger.info(f"Fetching latest file of {dataset_name} version {dataset_version}")

    api = OpenDataAPI(api_token=api_key)

    # sort the files in descending order and only retrieve the first file
    params = {"maxKeys": 4, "orderBy": "created", "sorting": "desc"}
    response = api.list_files(dataset_name, dataset_version, params)
    # print(response)
    if "error" in response:
        logger.error(f"Unable to retrieve list of files: {response['error']}")
        sys.exit(1)

    
    # Filter files that end with '00.tar'
    filtered_files = [f for f in response["files"] if f.get("filename").endswith("00.tar")]
    
    if not filtered_files:
        logger.error("No files ending with '00.tar' found")
        sys.exit(1)

    # Assuming files are already sorted by creation date in the response, get the latest
    latest_file = filtered_files[0].get("filename")
    logger.info(f"Latest file is: {latest_file}")

    # fetch the download url and download the file
    response = api.get_file_url(dataset_name, dataset_version, latest_file)
    download_file_from_temporary_download_url(response["temporaryDownloadUrl"], latest_file)


if __name__ == "__main__":
    main()

INFO:__main__:Fetching latest file of harmonie_arome_cy40_p1 version 0.2
INFO:__main__:Latest file is: harm40_v1_p1_2024053100.tar


KeyboardInterrupt: 

In [128]:

def unpack_tar_file(tar_path):
    # Create destination folder path
    dest_folder = os.path.join(os.path.dirname(tar_path), os.path.basename(tar_path).rsplit('.', 1)[0])
    os.makedirs(dest_folder, exist_ok=True)  # Ensure the destination folder exists
    
    # Extract all contents of file in destination folder path
    with tarfile.open(tar_path, "r") as tar:
        tar.extractall(path=dest_folder)

# Replace with the path to your tar file
tar_path = "harm40_v1_p1_2024052900.tar"
unpack_tar_file(tar_path)


In [142]:
# Eindhoven
eindhoven_lat = 51.4416
eindhoven_lon = 5.4697

# Folder grib files
grib_folder = r"C:\Users\20193362\Desktop\InterProject\InterdisciplinaryProject\energy_prediction\harm40_v1_p1_2024052900"

# List needed parameters, see code matrix KNMI
parameters = {"temperature": "11", "windU": "33", "windV": "34", "globalRadiation":  "117"}

# Initialize list to hold the data
data_list = []

# Loop over each file 
for file_name in os.listdir(grib_folder):
    if file_name.endswith('_GB'):
        grib_file = os.path.join(grib_folder, file_name)
        grbs = pygrib.open(grib_file)
        
        # Retrieve the lat/lon grid
        first_message = grbs.message(1)
        lats, lons = first_message.latlons()
        
        # Find the closest grid point
        distance = np.sqrt((lats - eindhoven_lat)**2 + (lons - eindhoven_lon)**2)
        min_index = distance.argmin()
        nearest_point_lat = lats.flat[min_index]
        nearest_point_lon = lons.flat[min_index]

        data_date = str(first_message.dataDate)  # Format: YYYYMMDD
        data_time = first_message.dataTime  # Format: HHMM
        
        # Create the base datetime object from dataDate and dataTime
        base_datetime = datetime.strptime(f"{data_date} {data_time:04d}", "%Y%m%d %H%M")
        step_range = float(first_message.stepRange)
        valid_datetime = base_datetime + timedelta(hours=step_range)
        
        # Initialize a dictionary to hold the data for this file
        data_dict = {
            'file_name': file_name,
            'datetime': valid_datetime #,
            #'latitude': nearest_point_lat,
            #'longitude': nearest_point_lon
        }
        
        # Extract data for each parameter
        for param_name in parameters:
            try:
                grb_message = grbs.select(parameterName=parameters[param_name])[0] # First instance
                eindhoven_value = grb_message.values.flat[min_index]
                data_dict[param_name] = eindhoven_value
            except (IndexError, ValueError):
                data_dict[param_name] = np.nan # When parameter is not found in grib file
        
        grbs.close()
        
        # Append dictionary to list
        data_list.append(data_dict)

# Convert list of dictionaries to DF
gribData = pd.DataFrame(data_list)
df = gribData.copy()

df['windSpeed'] = np.sqrt(df['windU']**2 + df['windV']**2)
df['temperature'] = df['temperature'] - 272.15

df['globalRadiation'] = df['globalRadiation'].fillna(0)
# Calculate the difference between consecutive rows
df['Q'] = df['globalRadiation'].diff()
df['Q'] = df ['Q'] / 3600

weather_df = df.copy()
# Get solar position for the dates / times
solpos_df = solarposition.get_solarposition(
    weather_df['datetime'], latitude=eindhoven_lat,
    longitude=eindhoven_lon, altitude=0,
    temperature=weather_df['temperature']
)
solpos_df.index = weather_df.index

# Method 'Erbs' to go from GHI to DNI and DHI
irradiance_df = irradiance.erbs(weather_df['Q'], solpos_df['zenith'], weather_df.index)
irradiance_df['ghi'] = weather_df['Q']

# Add DNI and DHI to weather_df
weather_df['dni'] = irradiance_df['dni']
weather_df['dhi'] = irradiance_df['dhi']

df = weather_df.copy()
df = df.drop(['windU', 'windV'], axis=1)
df = df[:-1]
df = df.fillna(0)

print(df)

                         file_name            datetime  temperature  \
0   HA40_N25_202405290000_00000_GB 2024-05-29 00:00:00    15.719385   
1   HA40_N25_202405290000_00100_GB 2024-05-29 01:00:00    15.551904   
2   HA40_N25_202405290000_00200_GB 2024-05-29 02:00:00    15.408838   
3   HA40_N25_202405290000_00300_GB 2024-05-29 03:00:00    15.307031   
4   HA40_N25_202405290000_00400_GB 2024-05-29 04:00:00    15.463770   
5   HA40_N25_202405290000_00500_GB 2024-05-29 05:00:00    15.528223   
6   HA40_N25_202405290000_00600_GB 2024-05-29 06:00:00    15.560327   
7   HA40_N25_202405290000_00700_GB 2024-05-29 07:00:00    15.804834   
8   HA40_N25_202405290000_00800_GB 2024-05-29 08:00:00    16.125879   
9   HA40_N25_202405290000_00900_GB 2024-05-29 09:00:00    15.691064   
10  HA40_N25_202405290000_01000_GB 2024-05-29 10:00:00    15.560693   
11  HA40_N25_202405290000_01100_GB 2024-05-29 11:00:00    15.911768   
12  HA40_N25_202405290000_01200_GB 2024-05-29 12:00:00    16.234033   
13  HA

In [143]:

# Group by date
df['date'] = df['datetime'].dt.date
df = df.drop(['file_name', 'datetime'], axis=1)
grouped = df.groupby('date')

# Aggregate hourly values into lists per day
daily_data = grouped.agg(list)
keeps = {
    'temperature':'temperature_sequence',
    'windSpeed':'wind_speed_sequence',
    'dni':'dni_sequence',
    'dhi':'dhi_sequence',
    'globalRadiation':'global_irradiance_sequence'
    }
daily_data=daily_data[keeps.keys()].rename(keeps,axis=1)
daily_data.index.name = 'date'


# Print the resulting DataFrame
print(daily_data)

                                         temperature_sequence  \
date                                                            
2024-05-29  [15.719384765625023, 15.551904296875023, 15.40...   
2024-05-30  [13.755883789062523, 13.068994140625023, 13.01...   

                                          wind_speed_sequence  \
date                                                            
2024-05-29  [0.0, 5.041999640192526, 5.007828251785629, 5....   
2024-05-30  [1.7717661479428932, 1.9176039515453165, 1.946...   

                                                 dni_sequence  \
date                                                            
2024-05-29  [0.0, 0.0, 0.0, 0.0, 0.00011847502907301761, 0...   
2024-05-30  [0.0, 0.0, 0.0, 0.0, 0.3752065591863343, 0.495...   

                                                 dhi_sequence  \
date                                                            
2024-05-29  [0.0, 1.4522480440426476e-14, -1.4522480440426...   
2024-05-30  [0.0, 0.0,

In [144]:
import os  
os.makedirs('energy_data', exist_ok=True)  
daily_data.to_csv('energy_data/forecastweather48h.csv')  